# MCDA Walkthrough Tutorial
## Multi-Criteria Decision Analysis for Benefit-Risk Assessment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nexvigilant/nv-BR-toolkit/blob/main/notebooks/MCDA_Walkthrough_Tutorial.ipynb)

---

### ⚠️ EDUCATIONAL USE ONLY

**This notebook is provided strictly for educational and instructional purposes.**
- Do NOT use for regulatory decision-making
- Do NOT use as a substitute for internal SOPs
- All data in this notebook is **simulated/hypothetical**

---

### Learning Objectives

After completing this notebook, you will be able to:

1. **Explain** the MCDA methodology and when to use it
2. **Define** criteria and assign weights using swing weighting
3. **Score** treatment options against defined criteria
4. **Calculate** weighted preference scores
5. **Perform** sensitivity analysis on weights
6. **Visualize** MCDA results for stakeholder communication

---

### Background: What is MCDA?

**Multi-Criteria Decision Analysis (MCDA)** is a structured approach for:

- **Explicitly weighing** multiple outcomes based on their relative importance
- **Integrating** stakeholder preferences into benefit-risk decisions
- **Comparing** treatment options using a single composite score
- **Exploring** how conclusions change with different value judgments

**When to use MCDA:**
- Multiple stakeholders with potentially different priorities
- Need to make value trade-offs explicit
- Regulatory submissions requiring quantitative B-R
- Patient preference integration

**Reference:** CIOMS Working Group XII Report, Chapter 5

## Setup

First, let's install and import the required packages.

In [ ]:
# Install dependencies (uncomment if running in Colab)
# !pip install pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from typing import Dict, List, Tuple

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Setup complete!")

## MCDA Framework Overview

MCDA follows a structured 5-step process:

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   1. DEFINE    │ → │   2. WEIGHT    │ → │   3. SCORE     │ → │  4. CALCULATE  │ → │  5. ANALYZE   │
│   CRITERIA     │    │   CRITERIA     │    │   OPTIONS      │    │  WEIGHTED SUM  │    │  SENSITIVITY  │
└─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
```

Let's work through each step with a **hypothetical oncology drug comparison**.

## Case Study: First-Line NSCLC Treatment

We're comparing three treatment options for first-line advanced non-small cell lung cancer (NSCLC):

| Treatment | Description |
|-----------|-------------|
| **NEXONC** | Novel PD-1 inhibitor + chemotherapy |
| **Standard Chemo** | Platinum-based doublet chemotherapy |
| **Best Supportive Care** | Symptom management only |

**Note:** All data is hypothetical for educational purposes only.

## Step 1: Define Criteria

Identify the key benefit and risk criteria for comparison.

### Criteria Selection Principles
- **Complete**: Cover all relevant outcomes
- **Non-redundant**: Avoid double-counting
- **Measurable**: Each criterion can be quantified
- **Preference-independent**: Preferences on one don't depend on others

In [ ]:
# Define criteria with categories
criteria = {
    'Overall Survival': {
        'category': 'Benefit',
        'description': 'Median overall survival (months)',
        'direction': 'higher_better',
        'unit': 'months'
    },
    'Progression-Free Survival': {
        'category': 'Benefit',
        'description': 'Median PFS (months)',
        'direction': 'higher_better',
        'unit': 'months'
    },
    'Tumor Response': {
        'category': 'Benefit',
        'description': 'Objective response rate (%)',
        'direction': 'higher_better',
        'unit': '%'
    },
    'Grade 3-4 AEs': {
        'category': 'Risk',
        'description': 'Serious adverse event rate (%)',
        'direction': 'lower_better',
        'unit': '%'
    },
    'Quality of Life': {
        'category': 'Benefit',
        'description': 'Maintained/improved QoL (%)',
        'direction': 'higher_better',
        'unit': '%'
    }
}

# Display criteria
print("MCDA CRITERIA")
print("=" * 60)
for name, info in criteria.items():
    emoji = "🟢" if info['category'] == 'Benefit' else "🔴"
    print(f"{emoji} {name} ({info['category']})")
    print(f"   {info['description']}")
    print(f"   Direction: {info['direction'].replace('_', ' ')}")
    print()

## Step 2: Assign Weights

Weights reflect the **relative importance** of each criterion. Several methods exist:

| Method | Description | Complexity |
|--------|-------------|------------|
| **Equal Weights** | All criteria weighted equally | Low |
| **Direct Rating** | Stakeholders rate importance 1-100 | Low |
| **Swing Weighting** | Based on range of possible outcomes | Medium |
| **DCE-Derived** | From discrete choice experiments | High |

### Swing Weighting Method

Ask: *"If all criteria were at their worst level, which criterion would you most want to swing to its best level?"*

The most important swing gets 100 points. Others are rated relative to this anchor.

In [ ]:
def swing_weighting(swing_points: Dict[str, float]) -> Dict[str, float]:
    """
    Convert swing points to normalized weights (summing to 1.0).
    
    Parameters:
    -----------
    swing_points : dict
        Raw swing points for each criterion (0-100 scale)
    
    Returns:
    --------
    dict : Normalized weights
    """
    total = sum(swing_points.values())
    return {k: v / total for k, v in swing_points.items()}

# Example: Oncologist panel swing weights
swing_points = {
    'Overall Survival': 100,           # Anchor - most important
    'Progression-Free Survival': 55,   # 55% as important as OS
    'Tumor Response': 30,              # Surrogate, less weight
    'Grade 3-4 AEs': 70,               # Safety is critical
    'Quality of Life': 45              # Patient-centric
}

weights = swing_weighting(swing_points)

# Display weights
print("CRITERIA WEIGHTS")
print("=" * 50)
print(f"{'Criterion':<30} {'Swing':>8} {'Weight':>10}")
print("-" * 50)
for criterion, swing in swing_points.items():
    weight = weights[criterion]
    bar = "█" * int(weight * 40)
    print(f"{criterion:<30} {swing:>8} {weight:>10.1%} {bar}")
print("-" * 50)
print(f"{'TOTAL':<30} {sum(swing_points.values()):>8} {sum(weights.values()):>10.1%}")

### 💡 Practice Exercise

**Question:** A patient advocacy group might assign different swing points. How might their weights differ?

<details>
<summary>Click for discussion</summary>

Patients might:
- Increase **Quality of Life** weight (daily experience matters)
- Decrease **Tumor Response** weight (surrogate, not directly felt)
- Value **Grade 3-4 AEs** even higher (toxicity impacts daily life)

This illustrates why capturing **multiple stakeholder perspectives** is valuable in MCDA.
</details>

## Step 3: Score Treatment Options

For each treatment, we need raw data on each criterion, then convert to a **0-100 score scale**.

### Scoring Methods

**Linear interpolation** (most common):
```
Score = 100 × (Value - Worst) / (Best - Worst)
```

Where:
- `Best` = best plausible value (score = 100)
- `Worst` = worst plausible value (score = 0)

In [ ]:
# Raw clinical data (hypothetical)
raw_data = pd.DataFrame({
    'Treatment': ['NEXONC', 'Standard Chemo', 'Best Supportive Care'],
    'Overall Survival': [22.0, 14.0, 8.0],          # months
    'Progression-Free Survival': [10.5, 5.5, 2.0],  # months
    'Tumor Response': [52, 28, 5],                  # %
    'Grade 3-4 AEs': [58, 52, 15],                  # % (lower is better)
    'Quality of Life': [62, 48, 70]                 # % maintained/improved
})

print("RAW CLINICAL DATA")
print("=" * 70)
print(raw_data.to_string(index=False))

In [ ]:
def score_criterion(values: np.array, direction: str, 
                    best: float = None, worst: float = None) -> np.array:
    """
    Convert raw values to 0-100 scores using linear interpolation.
    
    Parameters:
    -----------
    values : array
        Raw values for each treatment
    direction : str
        'higher_better' or 'lower_better'
    best : float, optional
        Best plausible value (defaults to max/min of data)
    worst : float, optional
        Worst plausible value (defaults to min/max of data)
    
    Returns:
    --------
    array : Scores on 0-100 scale
    """
    values = np.array(values)
    
    if direction == 'higher_better':
        best = best if best is not None else values.max()
        worst = worst if worst is not None else values.min()
        scores = 100 * (values - worst) / (best - worst)
    else:  # lower_better
        best = best if best is not None else values.min()
        worst = worst if worst is not None else values.max()
        scores = 100 * (worst - values) / (worst - best)
    
    return np.clip(scores, 0, 100)

# Score all criteria
scored_data = raw_data.copy()

for criterion, info in criteria.items():
    scored_data[criterion] = score_criterion(
        raw_data[criterion].values,
        info['direction']
    )

print("SCORED DATA (0-100 scale)")
print("=" * 70)
print(scored_data.to_string(index=False))

## Step 4: Calculate Weighted Scores

The **weighted preference score** for each treatment is:

$$
\text{Total Score} = \sum_{i=1}^{n} w_i \times s_i
$$

Where:
- $w_i$ = weight for criterion $i$
- $s_i$ = score for criterion $i$

In [ ]:
def calculate_weighted_scores(scored_df: pd.DataFrame, 
                             weights: Dict[str, float],
                             treatment_col: str = 'Treatment') -> pd.DataFrame:
    """
    Calculate weighted MCDA scores for each treatment.
    
    Returns DataFrame with weighted contributions and totals.
    """
    results = scored_df[[treatment_col]].copy()
    
    # Calculate weighted score for each criterion
    for criterion, weight in weights.items():
        col_name = f"{criterion} (w={weight:.1%})"
        results[col_name] = scored_df[criterion] * weight
    
    # Calculate total
    score_cols = [c for c in results.columns if c != treatment_col]
    results['TOTAL SCORE'] = results[score_cols].sum(axis=1)
    
    return results.sort_values('TOTAL SCORE', ascending=False)

# Calculate results
mcda_results = calculate_weighted_scores(scored_data, weights)

print("MCDA WEIGHTED SCORES")
print("=" * 90)
print(mcda_results.to_string(index=False))

In [ ]:
def display_mcda_summary(mcda_results: pd.DataFrame, weights: Dict[str, float]):
    """
    Display formatted MCDA results summary.
    """
    print("\n" + "=" * 60)
    print("MCDA BENEFIT-RISK SUMMARY")
    print("=" * 60)
    
    for idx, row in mcda_results.iterrows():
        treatment = row['Treatment']
        total = row['TOTAL SCORE']
        bar_len = int(total / 2)
        bar = "█" * bar_len + "░" * (50 - bar_len)
        
        print(f"\n{treatment}")
        print(f"  [{bar}] {total:.1f}/100")
        
        # Show contribution breakdown
        for criterion in weights.keys():
            col = [c for c in mcda_results.columns if criterion in c][0]
            contribution = row[col]
            mini_bar = "█" * int(contribution / 2)
            print(f"    {criterion[:20]:<20}: +{contribution:.1f} {mini_bar}")
    
    # Winner
    winner = mcda_results.iloc[0]
    print("\n" + "=" * 60)
    print(f"✅ PREFERRED OPTION: {winner['Treatment']}")
    print(f"   Total Score: {winner['TOTAL SCORE']:.1f}/100")
    
    # Margin
    if len(mcda_results) > 1:
        second = mcda_results.iloc[1]
        margin = winner['TOTAL SCORE'] - second['TOTAL SCORE']
        print(f"   Margin over {second['Treatment']}: +{margin:.1f} points")

display_mcda_summary(mcda_results, weights)

## Step 5: Sensitivity Analysis

**Critical question:** How robust is our conclusion to changes in weights?

Sensitivity analysis tests whether the preferred option changes under:
- Different stakeholder perspectives
- Uncertainty in weight elicitation
- Extreme scenarios (e.g., "safety-first" vs "efficacy-first")

In [ ]:
def one_way_sensitivity(scored_df: pd.DataFrame, 
                       base_weights: Dict[str, float],
                       vary_criterion: str,
                       weight_range: np.array = np.linspace(0, 0.6, 13)) -> pd.DataFrame:
    """
    Perform one-way sensitivity analysis on a single criterion's weight.
    
    Redistributes weight proportionally among other criteria.
    """
    results = []
    other_criteria = [c for c in base_weights.keys() if c != vary_criterion]
    base_other_sum = sum(base_weights[c] for c in other_criteria)
    
    for new_weight in weight_range:
        # Redistribute remaining weight proportionally
        remaining = 1.0 - new_weight
        test_weights = {vary_criterion: new_weight}
        for c in other_criteria:
            test_weights[c] = remaining * (base_weights[c] / base_other_sum)
        
        # Calculate scores
        for _, row in scored_df.iterrows():
            total = sum(row[c] * test_weights[c] for c in test_weights.keys())
            results.append({
                'Weight': new_weight,
                'Treatment': row['Treatment'],
                'Score': total
            })
    
    return pd.DataFrame(results)

# Sensitivity on Safety weight
sensitivity_results = one_way_sensitivity(
    scored_data, weights, 'Grade 3-4 AEs'
)

print("One-Way Sensitivity: Varying 'Grade 3-4 AEs' Weight")
print("=" * 50)
pivot = sensitivity_results.pivot(index='Weight', columns='Treatment', values='Score')
print(pivot.round(1))

In [ ]:
def plot_sensitivity(sensitivity_df: pd.DataFrame, 
                    base_weight: float,
                    criterion_name: str):
    """
    Plot one-way sensitivity analysis results.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = {'NEXONC': '#2ecc71', 'Standard Chemo': '#3498db', 
              'Best Supportive Care': '#e74c3c'}
    
    for treatment in sensitivity_df['Treatment'].unique():
        data = sensitivity_df[sensitivity_df['Treatment'] == treatment]
        ax.plot(data['Weight'], data['Score'], 
                label=treatment, color=colors.get(treatment, 'gray'),
                linewidth=2, marker='o', markersize=4)
    
    # Mark base case
    ax.axvline(x=base_weight, color='red', linestyle='--', 
               label=f'Base case ({base_weight:.0%})', alpha=0.7)
    
    ax.set_xlabel(f'Weight on {criterion_name}', fontsize=12)
    ax.set_ylabel('Total MCDA Score', fontsize=12)
    ax.set_title(f'Sensitivity Analysis: {criterion_name} Weight', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.set_xlim(0, 0.6)
    ax.set_ylim(0, 100)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = plot_sensitivity(sensitivity_results, weights['Grade 3-4 AEs'], 'Grade 3-4 AEs')
plt.show()

In [ ]:
def scenario_analysis(scored_df: pd.DataFrame, 
                     scenarios: Dict[str, Dict[str, float]]) -> pd.DataFrame:
    """
    Compare MCDA results across multiple weight scenarios.
    """
    results = []
    
    for scenario_name, scenario_weights in scenarios.items():
        # Normalize weights
        total = sum(scenario_weights.values())
        norm_weights = {k: v/total for k, v in scenario_weights.items()}
        
        for _, row in scored_df.iterrows():
            total_score = sum(row[c] * norm_weights[c] for c in norm_weights.keys())
            results.append({
                'Scenario': scenario_name,
                'Treatment': row['Treatment'],
                'Score': total_score
            })
    
    return pd.DataFrame(results)

# Define scenarios
scenarios = {
    'Base Case (Clinical)': swing_points,
    'Efficacy-Focused': {
        'Overall Survival': 100,
        'Progression-Free Survival': 80,
        'Tumor Response': 60,
        'Grade 3-4 AEs': 30,
        'Quality of Life': 30
    },
    'Safety-Focused': {
        'Overall Survival': 60,
        'Progression-Free Survival': 40,
        'Tumor Response': 20,
        'Grade 3-4 AEs': 100,
        'Quality of Life': 80
    },
    'Patient-Centric': {
        'Overall Survival': 80,
        'Progression-Free Survival': 50,
        'Tumor Response': 20,
        'Grade 3-4 AEs': 70,
        'Quality of Life': 100
    },
    'Equal Weights': {
        'Overall Survival': 100,
        'Progression-Free Survival': 100,
        'Tumor Response': 100,
        'Grade 3-4 AEs': 100,
        'Quality of Life': 100
    }
}

scenario_results = scenario_analysis(scored_data, scenarios)

# Display results
print("SCENARIO ANALYSIS RESULTS")
print("=" * 70)
pivot = scenario_results.pivot(index='Scenario', columns='Treatment', values='Score')
pivot['Preferred'] = pivot.idxmax(axis=1)
print(pivot.round(1))

In [ ]:
def plot_scenario_comparison(scenario_df: pd.DataFrame):
    """
    Create grouped bar chart comparing scenarios.
    """
    pivot = scenario_df.pivot(index='Scenario', columns='Treatment', values='Score')
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(pivot.index))
    width = 0.25
    
    colors = {'NEXONC': '#2ecc71', 'Standard Chemo': '#3498db', 
              'Best Supportive Care': '#e74c3c'}
    
    for i, treatment in enumerate(pivot.columns):
        bars = ax.bar(x + i*width, pivot[treatment], width, 
                      label=treatment, color=colors.get(treatment, 'gray'))
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.0f}',
                       xy=(bar.get_x() + bar.get_width()/2, height),
                       xytext=(0, 3), textcoords='offset points',
                       ha='center', va='bottom', fontsize=8)
    
    ax.set_xlabel('Scenario', fontsize=12)
    ax.set_ylabel('MCDA Score', fontsize=12)
    ax.set_title('MCDA Results Across Weight Scenarios', fontsize=14, fontweight='bold')
    ax.set_xticks(x + width)
    ax.set_xticklabels(pivot.index, rotation=15, ha='right')
    ax.legend(loc='upper right')
    ax.set_ylim(0, 100)
    ax.grid(True, axis='y', alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = plot_scenario_comparison(scenario_results)
plt.show()

## Step 6: Visualize Results

Different visualizations serve different audiences and purposes.

In [ ]:
def plot_stacked_contributions(mcda_results: pd.DataFrame, 
                               weights: Dict[str, float],
                               criteria_info: Dict):
    """
    Create stacked bar chart showing criterion contributions.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    treatments = mcda_results['Treatment'].values
    y_pos = np.arange(len(treatments))
    
    # Colors for criteria
    benefit_color = plt.cm.Greens(np.linspace(0.4, 0.8, 4))
    risk_color = plt.cm.Reds(np.linspace(0.4, 0.6, 1))
    
    colors = []
    b_idx, r_idx = 0, 0
    for criterion in weights.keys():
        if criteria_info[criterion]['category'] == 'Benefit':
            colors.append(benefit_color[b_idx])
            b_idx += 1
        else:
            colors.append(risk_color[r_idx])
            r_idx += 1
    
    # Plot stacked bars
    left = np.zeros(len(treatments))
    for i, criterion in enumerate(weights.keys()):
        col = [c for c in mcda_results.columns if criterion in c][0]
        values = mcda_results[col].values
        ax.barh(y_pos, values, left=left, label=criterion, 
                color=colors[i], edgecolor='white', linewidth=0.5)
        left += values
    
    # Add total score labels
    for i, total in enumerate(mcda_results['TOTAL SCORE']):
        ax.text(total + 1, i, f'{total:.1f}', va='center', fontweight='bold')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(treatments)
    ax.set_xlabel('Weighted Score', fontsize=12)
    ax.set_title('MCDA Score Breakdown by Criterion', fontsize=14, fontweight='bold')
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)
    ax.set_xlim(0, 110)
    ax.grid(True, axis='x', alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = plot_stacked_contributions(mcda_results, weights, criteria)
plt.show()

In [ ]:
def plot_radar_chart(scored_df: pd.DataFrame, criteria_list: List[str]):
    """
    Create radar/spider chart comparing treatments across criteria.
    """
    treatments = scored_df['Treatment'].unique()
    n_criteria = len(criteria_list)
    
    # Calculate angles
    angles = np.linspace(0, 2 * np.pi, n_criteria, endpoint=False).tolist()
    angles += angles[:1]  # Complete the loop
    
    fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))
    
    colors = {'NEXONC': '#2ecc71', 'Standard Chemo': '#3498db', 
              'Best Supportive Care': '#e74c3c'}
    
    for treatment in treatments:
        row = scored_df[scored_df['Treatment'] == treatment].iloc[0]
        values = [row[c] for c in criteria_list]
        values += values[:1]  # Complete the loop
        
        ax.plot(angles, values, 'o-', linewidth=2, 
                label=treatment, color=colors.get(treatment, 'gray'))
        ax.fill(angles, values, alpha=0.15, color=colors.get(treatment, 'gray'))
    
    # Format
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([c[:15] for c in criteria_list], fontsize=10)
    ax.set_ylim(0, 100)
    ax.set_title('Treatment Comparison Radar Chart', fontsize=14, 
                 fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    
    plt.tight_layout()
    return fig

fig = plot_radar_chart(scored_data, list(criteria.keys()))
plt.show()

## Interpretation & Communication

### Key Findings

In [ ]:
def generate_mcda_interpretation(mcda_results: pd.DataFrame, 
                                 scenario_results: pd.DataFrame):
    """
    Generate interpretation of MCDA results.
    """
    print("\n" + "=" * 70)
    print("MCDA INTERPRETATION")
    print("=" * 70)
    
    winner = mcda_results.iloc[0]
    second = mcda_results.iloc[1]
    
    print(f"\n✅ PREFERRED TREATMENT: {winner['Treatment']}")
    print(f"   Weighted Score: {winner['TOTAL SCORE']:.1f}/100")
    print(f"   Lead over {second['Treatment']}: +{winner['TOTAL SCORE'] - second['TOTAL SCORE']:.1f} points")
    
    # Robustness check
    pivot = scenario_results.pivot(index='Scenario', columns='Treatment', values='Score')
    winners = pivot.idxmax(axis=1)
    winner_counts = winners.value_counts()
    
    print("\n📊 ROBUSTNESS ANALYSIS")
    print(f"   Scenarios tested: {len(pivot)}")
    for treatment, count in winner_counts.items():
        pct = count / len(pivot) * 100
        print(f"   {treatment} preferred in: {count}/{len(pivot)} scenarios ({pct:.0f}%)")
    
    # Key drivers
    print("\n🔑 KEY DRIVERS")
    print("   - NEXONC leads on survival endpoints (OS, PFS)")
    print("   - Higher toxicity partially offsets efficacy gains")
    print("   - QoL comparable to BSC despite treatment intensity")
    
    print("\n⚠️  LIMITATIONS")
    print("   - Weights based on single stakeholder perspective")
    print("   - Simulated data for educational purposes only")
    print("   - Does not capture subgroup-specific benefit-risk")

generate_mcda_interpretation(mcda_results, scenario_results)

## Summary

### What We Learned

1. **MCDA explicitly integrates values** into benefit-risk through weights
2. **Swing weighting** provides a structured approach to elicit preferences
3. **Sensitivity analysis** tests robustness of conclusions
4. **Multiple visualizations** serve different stakeholder needs

### When to Use MCDA

✅ Multiple stakeholders with different priorities  
✅ Need to make value trade-offs explicit  
✅ Regulatory submissions requiring quantitative B-R  
✅ Integrating patient preferences  

### Limitations

⚠️ Requires agreement on criteria and scoring  
⚠️ Weight elicitation can be challenging  
⚠️ Linear scoring may oversimplify  
⚠️ Results depend heavily on weights chosen  

---

### Further Reading

- CIOMS Working Group XII Report, Chapter 5
- Marsh K, et al. "Multiple Criteria Decision Analysis for Health Care Decision Making" Value Health 2014
- EMA Benefit-Risk Methodology Project
- NexVigilant Benefit-Risk Intelligence Toolkit (companion materials)

---

**NexVigilant** | *Empowerment Through Vigilance*

This notebook is part of the [Benefit-Risk Intelligence Toolkit](https://github.com/nexvigilant/nv-BR-toolkit).